In [1]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
%load_ext autoreload

In [3]:
from pathlib import Path
import pandas as pd

In [4]:
import  datatset
from torch.utils.data import DataLoader

In [5]:
# DataLoader

In [6]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [7]:
num_calsses = len(df["category_id"].value_counts())

In [8]:
# df["category_id"]
# sample = df_train.groupby('category_id').apply(lambda x: x.sample(n=20, replace=True)).reset_index(drop = True)

In [9]:
# len(sample)// 64

In [10]:
import numpy

In [11]:
ds = datatset.HerbDataSet(df_train, TRAIN_PATH,256,y_name="category_id")

In [12]:
x,y = ds[0]
y

tensor(15672, device='cuda:0')

In [13]:
data_loader = DataLoader(ds, batch_size=32, shuffle=False)

In [14]:
# Model

In [15]:
import torchvision.models as models

In [16]:
import torch.nn as nn
import torch

In [17]:
base = models.resnet34(pretrained=True)

In [18]:
base.conv1

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [19]:
head = nn.Linear(in_features=512, out_features=num_calsses)

In [20]:
base.fc = head

In [21]:
base.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [22]:
# Tracking

In [23]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter("/home/shaun/personal/kaggle-data/nybg2020/logs/run-4")

In [24]:
import metrics
import train

In [25]:
acc =  metrics.Accuracy()
manager = metrics.MetricManager([acc], writer)

In [26]:
# Train

In [27]:
criterion = nn.CrossEntropyLoss()

In [28]:
layer_lrs = [
    {'params': base.layer1.parameters(), 'lr': 1e-4},
    {'params': base.layer2.parameters(), 'lr': 1e-4},
    {'params': base.layer3.parameters(), 'lr': 1e-3},
    {'params': base.layer4.parameters(), 'lr': 1e-3},
    {'params': base.fc.parameters(), 'lr': 1e-2}
]

In [29]:

optimizer = torch.optim.Adam(layer_lrs)

In [30]:
model_path="/home/shaun/personal/kaggle-data/nybg2020/models/"

In [31]:
trainer = train.Training(manager, criterion, optimizer, data_loader, 3, base, model_path )

In [32]:
! export CUDA_LAUNCH_BLOCKING=1

In [33]:
trainer.train_loop()

Epoch 0/3
step failed at: 23875


KeyboardInterrupt: 